# JAX 快速入门

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/jax/blob/main/docs/notebooks/quickstart.ipynb)

**JAX是可以在CPU、GPU和TPU上运行的NumPy，并且支持自动微分（automatic differentiation），可用于机器学习领域。**

作为[Autograd](https://github.com/hips/autograd)项目的升级版，JAX可以对原始Python以及Numpy代码进行自动微分计算。 JAX的的自动微分功能非常强大：
1. 支持大量Python语法特性，包括循环（loop）、条件判断（if-else）、递归（recursion）和闭包（closure）；
2. 支持高阶求导；
3. 支持reverse-mode和forward-mode两种微分方式，并且二者可以任意组合。

JAX依赖[XLA](https://www.tensorflow.org/xla)编译代码，然后在加速卡上执行，比如GPU和TPU。默认情况下，用户不需要关注编译过程，后台自动进行JIT编译并执行，但是JAX支持用户调用JIT(just-in-time)编译自己的Python函数得到XLA优化过的kernel。编译和自动微分可以任意组合，使得你在Python环境下就可以实现性能卓越的复杂算法。

In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

## 矩阵乘法

下面👇🏻的例子中，我们会生成一些随机数。你将会看到，在如何生成随机数这个问题上，Numpy和JAX的处理方式很不同，想了解更多细节可以查看 [Common Gotchas in JAX].

[Common Gotchas in JAX]: https://jax.readthedocs.io/en/latest/notebooks/Common_Gotchas_in_JAX.html#%F0%9F%94%AA-Random-Numbers

In [2]:
key = random.PRNGKey(0)
x = random.normal(key, (10,))
print(x)

[-0.3721097   0.2642309  -0.18252775 -0.7368085  -0.44030353 -0.15214416
 -0.6713451  -0.590867    0.7316775   0.567302  ]


看一下两个矩阵相乘的例子

In [3]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()  # runs on the TPU

4.08 ms ± 95.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


我们添加了 `block_until_ready` ，是因为JAX默认使用异步执行。

在Numpy array上使用JAX中的函数：

In [4]:
import numpy as np
x = np.random.normal(size=(size, size)).astype(np.float32)
%timeit jnp.dot(x, x.T).block_until_ready()

37.5 ms ± 425 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


可以看到上面的代码执行比较慢，因为它需要把数据传输到TPU，你可以手动调用`jax.device_put`将NDArray传输到TPU。

In [5]:
from jax import device_put

x = np.random.normal(size=(size, size)).astype(np.float32)
x = device_put(x)
%timeit jnp.dot(x, x.T).block_until_ready()

4.01 ms ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
print(type(x))

<class 'jaxlib.xla_extension.DeviceArray'>


`jax.device_put`的输出就像NDArray一样，但是它的类型已经成了DeviceArray，只有当你打印它的值或者保存到硬盘等操作时，JAX会将它的值复制到CPU。 `jax.device_put(x)`和`jit(lambda x: x)(x)`功能相同，但是前者速度更快。

如果你有加速卡，比如GPU或TPU，这些代码会在加速卡上运行，通常比CPU上快很多。

JAX可不仅仅是支持GPU的Numpy那么简单，它还包含了很多有用的函数转换功能，重要的有限三个:

 - `jax.jit`, 用于加速你的代码
 - `jax.grad`, 用于计算微分
 - `jax.vmap`, 用于自动矢量化或者批处理（batching）

让我们一个一个来介绍，最后再看下如何将他们组合发挥更大的作用。

## 使用`jax.jit`来加速函数执行

JAX自动将代码在GPU或TPU上执行，除非这俩都没有，才在CPU上执行。但是，上面的那些例子，JAX每次只向TPU分配一个计算操作，如果我们有多个计算操作，可以使用`@jit`修饰符借助[XLA](https://www.tensorflow.org/xla)将这些计算操作一起编译。

In [7]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

1.33 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


我们使用`@jit`进行加速，当第一次调用`selu`时会进行jit编译然后将编译结果缓存供后续调用使用。

In [8]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

274 µs ± 3.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## 使用`jax.grad`计算微分

除了使用数值型函数进行求值，我们还可以对它们进行转换（transform），其中一个转换就是[自动微分（automatic differentiation）](https://en.wikipedia.org/wiki/Automatic_differentiation)。和[Autograd](https://github.com/HIPS/autograd)一样，在JAX中，只需要使用`jax.grad`就可以得到导函数。

In [9]:
def sum_logistic(x):
  return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

x_small = jnp.arange(3.)
derivative_fn = grad(sum_logistic)  # 导函数
print(derivative_fn(x_small))

[0.25       0.1966118  0.10499343]


让我们验证下计算结果。

In [10]:
def first_finite_differences(f, x):
  eps = 1e-3
  return jnp.array([(f(x + eps * v) - f(x - eps * v)) / (2 * eps)
                   for v in jnp.eye(len(x))])


print(first_finite_differences(sum_logistic, x_small))

[0.24974345 0.1965761  0.10490417]


`jax.grad` 和`jax.jit` 可以任意组合，比如：

In [11]:
print(grad(jit(grad(jit(grad(sum_logistic)))))(1.0))

-0.03532532


对于一些复杂自动微分操作，你还可以使用`jax.vjp`进行 reverse-mode vector-Jacobian products，`jax.jvp` for forward-mode Jacobian-vector products。 这两个函数也可以任意组合，以及搭配其他JAX转换，比如我们看一个计算Hessian矩阵的例子:

In [12]:
from jax import jacfwd, jacrev
def hessian(fun):
  return jit(jacfwd(jacrev(fun)))

## 使用`jax.vmap`自动矢量化

JAX另一个很有用的转换是`jax.vmap`，也就是矢量化的map。 比起for循环+map，它的速度更快，当搭配`jax.jit`效果更佳。

我们来看一个简单的例子，用`jax.vmap`将矩阵-向量乘法扩展为矩阵-矩阵乘法。

In [13]:
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))

def apply_matrix(v):
  return jnp.dot(mat, v)

`apply_matrix`是一个矩阵-向量乘法函数，为了实现矩阵-矩阵乘法，我们可以在batch维度进行for循环，但是这样做效率会很差： 

In [14]:
def naively_batched_apply_matrix(v_batched):
  return jnp.stack([apply_matrix(v) for v in v_batched])

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

Naively batched
2.87 ms ± 317 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


我们知道`jnp.dot`本身支持矩阵-矩阵乘法，所以我们可以重新写一个矩阵乘法函数`batched_apply_matrix`：

In [ ]:
@jit
def batched_apply_matrix(v_batched):
  return jnp.dot(v_batched, mat.T)

print('Manually batched')
%timeit batched_apply_matrix(batched_x).block_until_ready()

Manually batched


上面的例子很简单，我们重新实现一个函数没问题，但是如果`apply_matrix`本身非常复杂呢，再写一个`batched_apply_matrix`代价就会非常高，这时候就可以借助`jax.vmap`自动让`apply_matrix`支持批处理（batch）。

In [ ]:
@jit
def vmap_batched_apply_matrix(v_batched):
  return vmap(apply_matrix)(v_batched)

print('Auto-vectorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

同样地，`jax.vmap`可以和其他JAX转换任意组合搭配。